In [1]:
#importing libraries
import torch
import numpy as np
import json
import re
from collections import defaultdict, Counter
from typing import Dict, List, Tuple, Any, Optional
import heapq
from dataclasses import dataclass

# For embeddings and similarity
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# For sentiment analysis
from transformers import pipeline as hf_pipeline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from huggingface_hub import login
login(token="")

In [3]:
def load_finetuned_model_from_drive():
    """Load the fine-tuned LLaMA model from Google Drive"""
    from transformers import AutoTokenizer, AutoModelForCausalLM


    model_path = "/content/drive/MyDrive/1B20EP/llama32_1b_outline_trained"

    try:
        # Try to load with the fix_mistral_regex flag
        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            fix_mistral_regex=True  # Add this to fix the warning
        )
        model = AutoModelForCausalLM.from_pretrained(model_path)

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        print(f"✅ Model loaded successfully from {model_path}")
        return tokenizer, model

    except Exception as e:
        print(f"⚠️ Could not load fine-tuned model: {e}")
        print("Using a smaller pre-trained model as fallback...")

        # Fallback to a model that definitely works
        tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
        model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
        tokenizer.pad_token = tokenizer.eos_token
        return tokenizer, model

In [4]:
tokenizer, model = load_finetuned_model_from_drive()


⚠️ Could not load fine-tuned model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/1B20EP/llama32_1b_outline_trained'. Use `repo_type` argument if needed.
Using a smaller pre-trained model as fallback...


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#story generate module
class StoryGenerator:
    """Module for generating stories using the fine-tuned model"""

    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def generate_story(self,
                      prompt: str = "Once upon a time",
                      max_length: int = 500,
                      temperature: float = 0.8,
                      top_p: float = 0.95,
                      do_sample: bool = True) -> str:
        """
        Generate a story from a prompt
        """
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=max_length,
                temperature=temperature,
                top_p=top_p,
                do_sample=do_sample,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                num_return_sequences=1
            )

        story = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return story

# Initialize generator
story_generator = StoryGenerator(tokenizer, model)

In [7]:
class NarrativeEvaluator:
    """Comprehensive narrative evaluation module"""

    def __init__(self):
        # Load embedding model for coherence scoring
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')

        # Load sentiment analysis pipeline
        self.sentiment_analyzer = hf_pipeline("sentiment-analysis")

        # Character tracking
        self.characters = {}

        # Cache for embeddings
        self.embedding_cache = {}

    def _get_embedding(self, text: str) -> np.ndarray:
        """Get or compute embedding for text"""
        if text not in self.embedding_cache:
            self.embedding_cache[text] = self.embedder.encode(text)
        return self.embedding_cache[text]

    def coherence_scoring(self, story: str) -> Dict:
        """
        2.1 Coherence Scoring
        Measure sentence connectivity and detect contradictions
        """
        # Split into sentences
        sentences = re.split(r'[.!?]+', story)
        sentences = [s.strip() for s in sentences if len(s.strip()) > 10]

        if len(sentences) < 2:
            return {"score": 0.5, "issues": ["Not enough sentences for coherence analysis"]}

        # Get embeddings for each sentence
        embeddings = [self._get_embedding(s) for s in sentences]

        # Compute pairwise cosine similarities
        similarities = []
        for i in range(len(embeddings) - 1):
            sim = cosine_similarity(
                embeddings[i].reshape(1, -1),
                embeddings[i + 1].reshape(1, -1)
            )[0][0]
            similarities.append(sim)

        avg_similarity = np.mean(similarities) if similarities else 0

        # Detect abrupt jumps (similarity < threshold)
        abrupt_jumps = []
        jump_threshold = 0.3
        for i, sim in enumerate(similarities):
            if sim < jump_threshold:
                abrupt_jumps.append({
                    "position": i,
                    "similarity": float(sim),
                    "sentences": [sentences[i], sentences[i+1]]
                })

        # Detect contradictions (simple pattern matching)
        contradictions = []
        negation_patterns = [r"not\s+\w+", r"never\s+\w+", r"no\s+\w+", r"but\s+.*\s+not"]
        for i, sentence in enumerate(sentences):
            for pattern in negation_patterns:
                if re.search(pattern, sentence.lower()):
                    contradictions.append({
                        "sentence": sentence,
                        "pattern": pattern
                    })

        # Score calculation
        base_score = float(avg_similarity)

        # Penalize abrupt jumps
        jump_penalty = len(abrupt_jumps) * 0.1
        base_score -= min(jump_penalty, 0.3)

        # Penalize contradictions
        contradiction_penalty = len(contradictions) * 0.05
        base_score -= min(contradiction_penalty, 0.2)

        # Ensure score is between 0 and 1
        score = max(0, min(1, base_score))

        return {
            "score": score,
            "average_similarity": float(avg_similarity),
            "abrupt_jumps": abrupt_jumps,
            "contradictions": contradictions
        }

    def emotional_trajectory_analysis(self, story: str) -> Dict:
        """
        2.2 Emotional Trajectory Analysis
        Analyze sentiment progression through the story
        """
        # Split into paragraphs
        paragraphs = [p.strip() for p in story.split('\n\n') if p.strip()]

        if len(paragraphs) < 2:
            return {"score": 0.5, "emotion_curve": []}

        # Analyze sentiment for each paragraph
        sentiments = []
        for para in paragraphs:
            try:
                result = self.sentiment_analyzer(para[:512])[0]  # Truncate if too long
                label = result['label']
                score = result['score']

                # Convert to numeric: POSITIVE=1, NEGATIVE=-1, NEUTRAL=0
                if label == 'POSITIVE':
                    sentiments.append(score)  # 0 to 1
                elif label == 'NEGATIVE':
                    sentiments.append(-score)  # -1 to 0
                else:
                    sentiments.append(0)
            except:
                sentiments.append(0)  # Neutral if analysis fails

        # Calculate trajectory stability
        if len(sentiments) > 1:
            changes = [abs(sentiments[i] - sentiments[i-1]) for i in range(1, len(sentiments))]
            avg_change = np.mean(changes) if changes else 0
        else:
            avg_change = 0

        # Score based on stability (lower changes are better)
        stability_score = max(0, 1 - avg_change)

        # Check for natural progression (not too flat, not too erratic)
        if len(sentiments) >= 3:
            # Calculate variance
            variance = np.var(sentiments)
            # Ideal variance is moderate (not 0, not too high)
            if variance < 0.05:  # Too flat
                progression_score = 0.6
            elif variance > 0.5:  # Too erratic
                progression_score = 0.6
            else:
                progression_score = 0.9
        else:
            progression_score = 0.7

        final_score = (stability_score + progression_score) / 2

        return {
            "score": float(final_score),
            "emotion_curve": sentiments,
            "stability_score": float(stability_score),
            "progression_score": float(progression_score)
        }

    def character_consistency_checks(self, story: str) -> Dict:
        """
        2.3 Character Consistency Checks
        Extract and track character attributes
        """
        # Simple character extraction using named entity patterns
        character_patterns = [
            r'(?:^|\s)([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?=\s+(?:said|asked|exclaimed|thought|was|were|had))',
            r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\'s',
            r'(?:^|\s)(?:Mr\.|Ms\.|Mrs\.|Dr\.)\s+([A-Z][a-z]+)',
        ]

        characters = {}
        inconsistencies = []

        # Extract character mentions
        for pattern in character_patterns:
            matches = re.findall(pattern, story)
            for match in matches:
                if isinstance(match, tuple):
                    match = match[0]
                char_name = match.strip()
                if len(char_name.split()) <= 3:  # Limit to reasonable names
                    if char_name not in characters:
                        characters[char_name] = {
                            'mentions': [],
                            'attributes': set(),
                            'actions': []
                        }

        # Track character mentions in sentences
        sentences = re.split(r'[.!?]+', story)
        for i, sentence in enumerate(sentences):
            sentence_lower = sentence.lower()
            for char_name in list(characters.keys()):
                if char_name.lower() in sentence_lower:
                    characters[char_name]['mentions'].append(i)

                    # Extract simple attributes
                    if 'brave' in sentence_lower or 'courageous' in sentence_lower:
                        characters[char_name]['attributes'].add('brave')
                    if 'smart' in sentence_lower or 'intelligent' in sentence_lower:
                        characters[char_name]['attributes'].add('smart')
                    if 'kind' in sentence_lower or 'gentle' in sentence_lower:
                        characters[char_name]['attributes'].add('kind')
                    if 'angry' in sentence_lower or 'mad' in sentence_lower:
                        characters[char_name]['attributes'].add('angry')

        # Check for inconsistencies
        for char_name, data in characters.items():
            mentions = data['mentions']
            if len(mentions) > 1:
                # Check for large gaps in character appearance
                gaps = [mentions[i] - mentions[i-1] for i in range(1, len(mentions))]
                if max(gaps) > 10:  # Large gap might indicate inconsistency
                    inconsistencies.append(f"Character '{char_name}' disappears for too long")

            # Check attribute consistency (simplified)
            attrs = list(data['attributes'])
            if 'brave' in attrs and 'cowardly' in sentence_lower:
                inconsistencies.append(f"Character '{char_name}' shows inconsistent bravery")

        # Calculate consistency score
        if characters:
            # Score based on number of inconsistencies
            inconsistency_penalty = min(len(inconsistencies) * 0.2, 0.8)
            base_score = 0.8 - inconsistency_penalty

            # Bonus for well-established characters
            well_established = sum(1 for c in characters.values() if len(c['mentions']) >= 3)
            if well_established >= 2:
                base_score += 0.1
        else:
            base_score = 0.5

        score = max(0, min(1, base_score))

        return {
            "score": score,
            "characters_detected": list(characters.keys()),
            "inconsistencies": inconsistencies,
            "character_details": {k: {"mentions": len(v['mentions']),
                                    "attributes": list(v['attributes'])}
                                for k, v in characters.items()}
        }

    def lexical_richness_score(self, story: str) -> Dict:
        """
        2.4 Lexical Richness Score
        Compute vocabulary diversity metrics
        """
        # Clean and tokenize
        words = re.findall(r'\b[a-zA-Z]{3,}\b', story.lower())

        if len(words) < 10:
            return {"score": 0.5, "ttr": 0, "repetitions": []}

        # Type-Token Ratio
        unique_words = set(words)
        ttr = len(unique_words) / len(words)

        # Detect repetitions
        word_counts = Counter(words)
        repeated_words = [(word, count) for word, count in word_counts.items()
                         if count > 3 and count > 0.05 * len(words)]

        # Check for diverse verbs and adjectives (simplified)
        # Common verbs and adjectives for comparison
        common_verbs = {'said', 'went', 'came', 'saw', 'looked', 'thought', 'felt'}
        common_adjs = {'big', 'small', 'good', 'bad', 'happy', 'sad', 'old', 'new'}

        # Count unique verbs and adjectives (simplified detection)
        verbs = [w for w in words if w.endswith(('ed', 'ing')) and len(w) > 4]
        adjs = [w for w in words if w.endswith(('y', 'ful', 'ous', 'ive')) and len(w) > 4]

        unique_verbs = set(verbs)
        unique_adjs = set(adjs)

        verb_diversity = len(unique_verbs) / max(len(verbs), 1)
        adj_diversity = len(unique_adjs) / max(len(adjs), 1)

        # Calculate score
        base_score = ttr * 0.6 + (verb_diversity * 0.2) + (adj_diversity * 0.2)

        # Penalize excessive repetition
        repetition_penalty = len(repeated_words) * 0.05
        base_score -= min(repetition_penalty, 0.3)

        score = max(0, min(1, base_score))

        return {
            "score": score,
            "ttr": ttr,
            "total_words": len(words),
            "unique_words": len(unique_words),
            "repeated_words": repeated_words[:5],  # Top 5 repeated
            "verb_diversity": verb_diversity,
            "adjective_diversity": adj_diversity
        }

    def evaluate_story_metrics(self, story: str) -> Dict:
        """
        Comprehensive evaluation of story metrics
        """
        coherence = self.coherence_scoring(story)
        emotion = self.emotional_trajectory_analysis(story)
        character = self.character_consistency_checks(story)
        lexical = self.lexical_richness_score(story)

        return {
            "coherence": coherence["score"],
            "emotion_curve": emotion["score"],
            "character_consistency": character["score"],
            "lexical_richness": lexical["score"],
            "details": {
                "coherence_details": coherence,
                "emotion_details": emotion,
                "character_details": character,
                "lexical_details": lexical
            }
        }

# Initialize evaluator
narrative_evaluator = NarrativeEvaluator()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [8]:
from typing import Dict, List

In [9]:

class BayesianReasoner:
    def __init__(self):
        # Prior probabilities
        self.prior_acceptable = 0.7  # P(Acceptable)
        self.prior_flawed = 0.3      # P(Flawed)

        # Conditional Probability Tables (handcrafted)
        self.cpts = {
            # P(Coherence | Narrative)
            'coherence': {
                'acceptable': {'high': 0.8, 'medium': 0.15, 'low': 0.05},
                'flawed': {'high': 0.1, 'medium': 0.3, 'low': 0.6}
            },
            # P(Emotion | Narrative)
            'emotion': {
                'acceptable': {'stable': 0.7, 'moderate': 0.2, 'erratic': 0.1},
                'flawed': {'stable': 0.2, 'moderate': 0.3, 'erratic': 0.5}
            },
            'character': {
                'acceptable': {'high': 0.8, 'medium': 0.15, 'low': 0.05},
                'flawed': {'high': 0.1, 'medium': 0.4, 'low': 0.5}
            }
        }

    def _discretize_score(self, score: float) -> str:
        """Convert continuous score to discrete category"""
        if score >= 0.8:
            return 'high'
        elif score >= 0.6:
            return 'medium'
        else:
            return 'low'

    def _discretize_emotion(self, score: float) -> str:
        """Discretize emotion score"""
        if score >= 0.8:
            return 'stable'
        elif score >= 0.6:
            return 'moderate'
        else:
            return 'erratic'

    def compute_acceptability(self, evaluation_scores: Dict) -> Dict:
        """
        Compute P(Acceptable | Evidence) using Bayes' rule
        """
        try:
            # Extract scores
            coherence_score = evaluation_scores.get('coherence', 0.5)
            emotion_score = evaluation_scores.get('emotion_curve', 0.5)
            character_score = evaluation_scores.get('character_consistency', 0.5)

            # Discretize scores
            coherence_cat = self._discretize_score(coherence_score)
            emotion_cat = self._discretize_emotion(emotion_score)
            character_cat = self._discretize_score(character_score)

            # Get likelihoods with safe access
            p_coherence_acceptable = self.cpts['coherence']['acceptable'].get(coherence_cat, 0.1)
            p_emotion_acceptable = self.cpts['emotion']['acceptable'].get(emotion_cat, 0.1)
            p_character_acceptable = self.cpts['character']['acceptable'].get(character_cat, 0.1)

            p_coherence_flawed = self.cpts['coherence']['flawed'].get(coherence_cat, 0.5)
            p_emotion_flawed = self.cpts['emotion']['flawed'].get(emotion_cat, 0.5)
            p_character_flawed = self.cpts['character']['flawed'].get(character_cat, 0.5)

            # Compute likelihood of evidence given each hypothesis
            # Assuming conditional independence
            likelihood_acceptable = (p_coherence_acceptable *
                                    p_emotion_acceptable *
                                    p_character_acceptable)

            likelihood_flawed = (p_coherence_flawed *
                                p_emotion_flawed *
                                p_character_flawed)

            # Compute marginal probability of evidence
            p_evidence = (likelihood_acceptable * self.prior_acceptable +
                          likelihood_flawed * self.prior_flawed)

            # Compute posterior probabilities
            if p_evidence > 0:
                p_acceptable_given_evidence = (likelihood_acceptable * self.prior_acceptable) / p_evidence
                p_flawed_given_evidence = (likelihood_flawed * self.prior_flawed) / p_evidence
            else:
                p_acceptable_given_evidence = 0.5
                p_flawed_given_evidence = 0.5

            # Decision threshold
            decision = "Acceptable" if p_acceptable_given_evidence >= 0.6 else "Flawed"

            return {
                "probability_acceptable": float(p_acceptable_given_evidence),
                "probability_flawed": float(p_flawed_given_evidence),
                "decision": decision,
                "evidence": {
                    "coherence": coherence_cat,
                    "emotion": emotion_cat,
                    "character": character_cat
                },
                "raw_scores": {
                    "coherence": coherence_score,
                    "emotion": emotion_score,
                    "character": character_score
                }
            }

        except Exception as e:
            print(f"⚠️ Error in Bayesian reasoning: {e}")
            # Return fallback result
            return {
                "probability_acceptable": 0.5,
                "probability_flawed": 0.5,
                "decision": "Uncertain",
                "evidence": {
                    "coherence": "unknown",
                    "emotion": "unknown",
                    "character": "unknown"
                }
            }

    def explain_decision(self, bayesian_result: Dict) -> str:
        """Generate human-readable explanation for the Bayesian decision"""
        if bayesian_result is None:
            return "⚠️ Bayesian analysis could not be completed. Story evaluation incomplete."

        try:
            # Safely get values with defaults
            decision = bayesian_result.get("decision", "Uncertain")
            prob = bayesian_result.get("probability_acceptable", 0.5)
            evidence = bayesian_result.get("evidence", {})

            explanations = []

            # Coherence explanation
            coherence = evidence.get("coherence", "unknown")
            if coherence == "low":
                explanations.append("The story has coherence issues with abrupt transitions.")
            elif coherence == "high":
                explanations.append("The story flows smoothly with good connections between ideas.")
            elif coherence == "medium":
                explanations.append("The story has decent coherence with some rough transitions.")

            # Emotion explanation
            emotion = evidence.get("emotion", "unknown")
            if emotion == "erratic":
                explanations.append("Emotional progression feels unnatural or jarring.")
            elif emotion == "stable":
                explanations.append("Emotional journey progresses naturally.")
            elif emotion == "moderate":
                explanations.append("Emotional flow is acceptable but could be smoother.")

            # Character explanation
            character = evidence.get("character", "unknown")
            if character == "low":  # Fixed: was "inconsistent"
                explanations.append("Characters show inconsistent traits or behavior.")
            elif character == "high":  # Fixed: was "consistent"
                explanations.append("Characters are well-developed and consistent.")
            elif character == "medium":  # Fixed: was "moderate"
                explanations.append("Character consistency is adequate but could be improved.")

            # Combine explanations (removed lexical section as it's not in compute_acceptability)
            if decision == "Acceptable":
                verdict = f"✅ Story is acceptable (confidence: {prob:.1%})"
            elif decision == "Flawed":
                verdict = f"❌ Story needs improvement (confidence: {prob:.1%})"
            else:
                verdict = f"⚠️ Story quality is uncertain (confidence: {prob:.1%})"

            if explanations:
                return verdict + "\n\nKey observations:\n- " + "\n- ".join(explanations)
            else:
                return verdict + "\n\nLimited evaluation data available."

        except Exception as e:
            return f"⚠️ Error explaining decision: {str(e)[:100]}"

    def get_recommendations(self, bayesian_result: Dict) -> List[str]:
        """Generate improvement recommendations based on weaknesses"""
        evidence = bayesian_result["evidence"]
        raw_scores = bayesian_result["raw_scores"]
        recommendations = []

        # Coherence recommendations
        if raw_scores["coherence"] < 0.6:
            recommendations.append("Improve story flow by adding transition sentences between paragraphs.")
            recommendations.append("Ensure each paragraph logically follows from the previous one.")

        # Emotion recommendations
        if raw_scores["emotion"] < 0.6:
            recommendations.append("Work on gradual emotional buildup rather than sudden changes.")
            recommendations.append("Add more emotional context to character decisions.")

        # Character recommendations
        if raw_scores["character"] < 0.6:
            recommendations.append("Establish clear character traits early and maintain them consistently.")
            recommendations.append("Give characters distinct voices and motivations.")

        return recommendations

In [10]:
class NarrativeIntelligenceSystem:
    """Main system integrating all components"""

    def __init__(self, tokenizer=None, model=None):
        # Initialize components
        if tokenizer and model:
            self.story_generator = StoryGenerator(tokenizer, model)
        else:
            # Load default if not provided
            tokenizer, model = load_finetuned_model_from_drive()
            self.story_generator = StoryGenerator(tokenizer, model)

        self.evaluator = NarrativeEvaluator()
        self.reasoner = BayesianReasoner()

    def generate_and_evaluate(self, prompt: str = "Once upon a time", max_length: int = 500) -> Dict:
        """Generate a story and evaluate it comprehensively"""
        print(f"📝 Generating story from prompt: '{prompt}'...")

        # 1. Generate story
        story = self.story_generator.generate_story(
            prompt=prompt,
            max_length=max_length
        )

        print(f"✅ Story generated ({len(story.split())} words)")
        print("\\n" + "="*50)
        print("📊 Evaluating narrative quality...")
        print("="*50)

        # 2. Evaluate story metrics
        evaluation = self.evaluator.evaluate_story_metrics(story)

        # 3. Bayesian reasoning
        print("\\n🤖 Applying Bayesian reasoning...")
        bayesian_result = self.reasoner.compute_acceptability(evaluation)

        # 4. Generate explanations
        explanation = self.reasoner.explain_decision(bayesian_result)
        recommendations = self.reasoner.get_recommendations(bayesian_result)

        # Compile final results
        result = {
            "story": story,
            "evaluation_scores": {
                "coherence": evaluation["coherence"],
                "emotion_curve": evaluation["emotion_curve"],
                "character_consistency": evaluation["character_consistency"],
                "lexical_richness": evaluation["lexical_richness"]
            },
            "bayesian_analysis": bayesian_result,
            "explanation": explanation,
            "recommendations": recommendations,
            "detailed_evaluation": evaluation["details"]
        }

        return result

    def print_report(self, result: Dict):
        """Print a formatted report of the analysis"""
        print("\\n" + "="*60)
        print("📋 NARRATIVE INTELLIGENCE SYSTEM - ANALYSIS REPORT")
        print("="*60)

        print(f"\\n📖 STORY (first 300 chars):")
        print(result["story"][:300] + "..." if len(result["story"]) > 300 else result["story"])

        print(f"\\n📊 EVALUATION SCORES:")
        scores = result["evaluation_scores"]
        for metric, score in scores.items():
            bar = "█" * int(score * 20) + "░" * (20 - int(score * 20))
            print(f"  {metric.replace('_', ' ').title():20} [{bar}] {score:.2f}")

        print(f"\\n🎯 BAYESIAN ANALYSIS:")
        bayes = result["bayesian_analysis"]
        print(f"  Probability Acceptable: {bayes['probability_acceptable']:.2%}")
        print(f"  Probability Flawed:     {bayes['probability_flawed']:.2%}")
        print(f"  Decision:               {bayes['decision']}")

        print(f"\\n💡 EXPLANATION:")
        print(result["explanation"])

        if result["recommendations"]:
            print(f"\\n🔧 RECOMMENDATIONS FOR IMPROVEMENT:")
            for i, rec in enumerate(result["recommendations"], 1):
                print(f"  {i}. {rec}")

        print("\\n" + "="*60)
        print("✅ ANALYSIS COMPLETE")
        print("="*60)

In [11]:
class ConstraintPlanner:
    """
    4. Constraint Planning Module
    Rule-based constraint enforcement
    """

    def __init__(self):
        # Define narrative constraints
        self.constraints = {
            "required_events": [
                {"name": "introduction", "description": "Characters and setting introduction"},
                {"name": "conflict", "description": "Main problem or challenge"},
                {"name": "climax", "description": "Turning point or peak tension"},
                {"name": "resolution", "description": "Problem resolution or conclusion"}
            ],
            "forbidden_contradictions": [
                {"pattern": ".*alive.*dead.*", "description": "Character cannot be both alive and dead"},
                {"pattern": ".*was.*but now.*", "description": "Abrupt contradictory statements"},
                {"pattern": ".*impossible.*happened.*", "description": "Violation of established rules"}
            ],
            "thematic_requirements": [
                {"theme": "hope", "required": True, "description": "Story should express hope by the end"}
            ],
            "world_rules": [
                {"rule": "magic_requires_cost", "description": "Magic should have consequences"},
                {"rule": "time_linear", "description": "Time should flow forward consistently"}
            ]
        }

        self.violations = []

    def detect_violations(self, story: str, evaluation_scores: Dict) -> List[Dict]:
        """
        4.2 Violation Detection Engine
        """
        self.violations = []

        # Check required events
        story_lower = story.lower()
        for event in self.constraints["required_events"]:
            event_name = event["name"]
            if event_name == "introduction":
                if not self._check_introduction(story_lower):
                    self.violations.append({
                        "type": "missing_event",
                        "event": event_name,
                        "severity": "high",
                        "description": f"Missing {event['description']}"
                    })
            elif event_name == "conflict":
                if not self._check_conflict(story_lower):
                    self.violations.append({
                        "type": "missing_event",
                        "event": event_name,
                        "severity": "high",
                        "description": f"Missing {event['description']}"
                    })
            elif event_name == "climax":
                if not self._check_climax(story_lower):
                    self.violations.append({
                        "type": "missing_event",
                        "event": event_name,
                        "severity": "medium",
                        "description": f"Missing or weak {event['description']}"
                    })
            elif event_name == "resolution":
                if not self._check_resolution(story_lower):
                    self.violations.append({
                        "type": "missing_event",
                        "event": event_name,
                        "severity": "high",
                        "description": f"Missing {event['description']}"
                    })

        # Check forbidden contradictions
        for contradiction in self.constraints["forbidden_contradictions"]:
            pattern = contradiction["pattern"]
            if re.search(pattern, story, re.IGNORECASE | re.DOTALL):
                self.violations.append({
                    "type": "contradiction",
                    "pattern": pattern,
                    "severity": "high",
                    "description": contradiction["description"]
                })

        # Check thematic requirements
        for theme_req in self.constraints["thematic_requirements"]:
            if theme_req["required"]:
                theme = theme_req["theme"]
                theme_words = {
                    "hope": ["hope", "hopeful", "optimistic", "better future", "renewed"],
                    "courage": ["brave", "courage", "bold", "fearless"],
                    "love": ["love", "affection", "caring", "compassion"]
                }

                if theme in theme_words:
                    found = any(word in story_lower for word in theme_words[theme])
                    if not found and "hopeless" not in story_lower and "despair" not in story_lower:
                        self.violations.append({
                            "type": "thematic_violation",
                            "theme": theme,
                            "severity": "medium",
                            "description": f"Missing expression of {theme}"
                        })

        # Check evaluation score violations
        if evaluation_scores["coherence"] < 0.6:
            self.violations.append({
                "type": "coherence_violation",
                "severity": "medium",
                "description": f"Low coherence score: {evaluation_scores['coherence']:.2f}"
            })

        if evaluation_scores["character_consistency"] < 0.5:
            self.violations.append({
                "type": "character_violation",
                "severity": "medium",
                "description": f"Low character consistency: {evaluation_scores['character_consistency']:.2f}"
            })

        return self.violations

    def _check_introduction(self, story: str) -> bool:
        """Check if introduction is present"""
        intro_indicators = ["once upon", "in a", "there was", "lived", "named", "introduction"]
        return any(indicator in story[:200].lower() for indicator in intro_indicators)

    def _check_conflict(self, story: str) -> bool:
        """Check if conflict is present"""
        conflict_indicators = ["but", "however", "problem", "challenge", "danger", "threat", "conflict"]
        return any(indicator in story for indicator in conflict_indicators)

    def _check_climax(self, story: str) -> bool:
        """Check if climax is present"""
        climax_indicators = ["suddenly", "finally", "realized", "turning point", "peak", "climax"]
        return any(indicator in story for indicator in climax_indicators)

    def _check_resolution(self, story: str) -> bool:
        """Check if resolution is present"""
        resolution_indicators = ["in the end", "finally", "resolution", "concluded", "happily ever after", "lesson learned"]
        return any(indicator in story[-200:].lower() for indicator in resolution_indicators)

    def repair_or_regenerate(self, story: str, violations: List[Dict]) -> str:
        """
        4.3 Repair or Regeneration Mechanism
        """
        if not violations:
            return story

        # Sort violations by severity
        severity_order = {"high": 3, "medium": 2, "low": 1}
        violations.sort(key=lambda x: severity_order.get(x.get("severity", "low"), 0), reverse=True)

        # Handle the most severe violation
        main_violation = violations[0]

        if main_violation["type"] == "missing_event":
            event_name = main_violation["event"]
            repair_prompt = self._create_repair_prompt(story, event_name)

            # Generate repaired segment
            repaired_segment = story_generator.generate_story(
                prompt=repair_prompt,
                max_length=200,
                temperature=0.7
            )

            # Integrate repaired segment
            if event_name == "resolution":
                # Append resolution
                repaired_story = story + "\n\n" + repaired_segment
            else:
                # Insert at appropriate place
                repaired_story = self._insert_event(story, event_name, repaired_segment)

            return repaired_story

        elif main_violation["type"] == "contradiction":
            # Rewrite to remove contradiction
            repair_prompt = f"Rewrite the following story segment to remove contradictions while maintaining the plot:\n\n{story}\n\nRevised version:"
            return story_generator.generate_story(
                prompt=repair_prompt,
                max_length=len(story) + 100,
                temperature=0.6
            )

        else:
            # For other violations, do minor repair
            repair_prompt = f"Improve the following story by fixing {main_violation['description']}:\n\n{story}\n\nImproved version:"
            return story_generator.generate_story(
                prompt=repair_prompt,
                max_length=len(story) + 50,
                temperature=0.7
            )

    def _create_repair_prompt(self, story: str, event_name: str) -> str:
        """Create prompt for repairing missing event"""
        prompts = {
            "introduction": f"Write an introduction for a story that sets up characters and setting:\n\n",
            "conflict": f"Write a conflict scene for this story:\n\n{story[:300]}...\n\nConflict:",
            "climax": f"Write a climax for this story:\n\n{story[:300]}...\n\nClimax:",
            "resolution": f"Write a resolution for this story:\n\n{story}\n\nResolution:"
        }
        return prompts.get(event_name, f"Write a {event_name} for the story")

    def _insert_event(self, story: str, event_name: str, event_text: str) -> str:
        """Insert event at appropriate position"""
        # Simple insertion logic
        paragraphs = story.split('\n\n')

        if event_name == "introduction" and len(paragraphs) > 1:
            paragraphs[0] = event_text + "\n\n" + paragraphs[0]
        elif event_name == "conflict" and len(paragraphs) > 2:
            insert_pos = min(2, len(paragraphs) - 1)
            paragraphs.insert(insert_pos, event_text)
        elif event_name == "climax" and len(paragraphs) > 3:
            insert_pos = len(paragraphs) // 2
            paragraphs.insert(insert_pos, event_text)

        return '\n\n'.join(paragraphs)

# Initialize constraint planner
constraint_planner = ConstraintPlanner()


In [12]:
class PlotSearch:
    """
    5. Heuristic Search for Plot Event Selection
    """

    def __init__(self):
        self.event_templates = [
            "Character discovers a secret",
            "Character faces a moral dilemma",
            "Unexpected obstacle appears",
            "Character learns an important lesson",
            "Relationship between characters deepens",
            "Character overcomes fear",
            "Major plot twist revealed",
            "Character makes a sacrifice",
            "Conflict escalates to confrontation",
            "Character achieves goal"
        ]

    def bfs_search(self, current_plot: str, target_theme: str, depth: int = 3) -> List[str]:
        """
        5.1 Uninformed Search (BFS)
        Explore possible next events
        """
        visited = set()
        queue = [(current_plot, [])]
        results = []

        while queue and len(results) < 5:
            plot, path = queue.pop(0)

            if len(path) >= depth:
                results.append((plot, path))
                continue

            # Generate next possible events
            for event in self.event_templates:
                new_plot = plot + f"\n\nEvent: {event}"
                new_path = path + [event]

                plot_hash = hash(new_plot[:100])
                if plot_hash not in visited:
                    visited.add(plot_hash)
                    queue.append((new_plot, new_path))

        return results[:3]  # Return top 3

    def a_star_search(self, current_plot: str, target_theme: str, max_steps: int = 5) -> List[str]:
        """
        5.2 Informed Search (A*)
        Use heuristic to find optimal next events
        """
        # Priority queue: (f_score, g_score, plot, path)
        open_set = []

        # Heuristic: similarity to target theme + low constraint violations
        start_state = (current_plot, [])
        g_score = {str(start_state): 0}
        f_score = {str(start_state): self._heuristic(current_plot, target_theme)}

        heapq.heappush(open_set, (f_score[str(start_state)], 0, current_plot, []))

        results = []

        while open_set and len(results) < 3:
            f, g, plot, path = heapq.heappop(open_set)

            if len(path) >= max_steps:
                results.append((plot, path))
                continue

            # Generate successors
            for event in self.event_templates:
                new_plot = plot + f"\n\nEvent: {event}"
                new_path = path + [event]
                new_state = (new_plot, new_path)
                state_key = str(new_state)

                # Calculate scores
                new_g = g + 1
                new_f = new_g + self._heuristic(new_plot, target_theme)

                if state_key not in g_score or new_g < g_score[state_key]:
                    g_score[state_key] = new_g
                    f_score[state_key] = new_f
                    heapq.heappush(open_set, (new_f, new_g, new_plot, new_path))

        return results

    def _heuristic(self, plot: str, target_theme: str) -> float:
        """
        Heuristic function for A* search
        Lower is better
        """
        # Heuristic 1: Theme similarity (simplified)
        theme_words = {
            "hope": ["hope", "hopeful", "optimistic"],
            "adventure": ["adventure", "journey", "explore"],
            "mystery": ["mystery", "secret", "clue"],
            "romance": ["love", "heart", "romance"]
        }

        plot_lower = plot.lower()
        theme_score = 0

        if target_theme in theme_words:
            target_words = theme_words[target_theme]
            matches = sum(1 for word in target_words if word in plot_lower)
            theme_score = 1.0 - (matches / len(target_words))

        # Heuristic 2: Estimated constraint violations
        # Simulate evaluation to estimate violations
        eval_scores = narrative_evaluator.evaluate_story_metrics(plot[:500])
        violation_score = 0

        if eval_scores["coherence"] < 0.6:
            violation_score += 0.3
        if eval_scores["character_consistency"] < 0.5:
            violation_score += 0.3

        # Combined heuristic
        total_heuristic = theme_score * 0.7 + violation_score * 0.3

        return total_heuristic

# Initialize plot search
plot_search = PlotSearch()

In [13]:
class RLRefinement:
    """
    6. Simplified Reinforcement Learning Refinement Loop
    """

    def __init__(self, quality_threshold: float = 0.7, max_iterations: int = 3):
        self.quality_threshold = quality_threshold
        self.max_iterations = max_iterations
        self.history = []

    def refine_story(self, story: str, evaluation_scores: Dict, violations: List[Dict]) -> Tuple[str, List[Dict]]:
        """
        RL-inspired refinement loop
        """
        iteration = 0
        current_story = story
        current_scores = evaluation_scores
        current_violations = violations
        actions_taken = []

        while iteration < self.max_iterations:
            iteration += 1

            # State: scores + violations
            state = {
                "scores": current_scores,
                "violation_count": len(current_violations),
                "iteration": iteration
            }

            # Determine action based on state
            action = self._select_action(state)
            actions_taken.append({
                "iteration": iteration,
                "action": action,
                "state": state.copy()
            })

            if action == "accept":
                print(f"  ✓ Iteration {iteration}: Accepting story (quality: {current_scores.get('overall', 0.5):.2f})")
                break

            elif action == "repair":
                print(f"  ↻ Iteration {iteration}: Repairing violations")
                # Repair using constraint planner
                current_story = constraint_planner.repair_or_regenerate(
                    current_story, current_violations
                )

                # Re-evaluate
                current_scores = narrative_evaluator.evaluate_story_metrics(current_story)
                current_violations = constraint_planner.detect_violations(
                    current_story, current_scores
                )

                # Calculate reward
                reward = self._calculate_reward(
                    old_scores=evaluation_scores,
                    new_scores=current_scores,
                    old_violations=len(violations),
                    new_violations=len(current_violations)
                )

                actions_taken[-1]["reward"] = reward

                if reward > 0:
                    print(f"    + Positive reward: {reward:.2f}")
                else:
                    print(f"    - Negative reward: {reward:.2f}")

            elif action == "regenerate":
                print(f"  ⟳ Iteration {iteration}: Regenerating story")
                # Generate new story with similar theme
                current_story = story_generator.generate_story(
                    prompt="Write a complete short story with: ",
                    max_length=500
                )

                # Re-evaluate
                current_scores = narrative_evaluator.evaluate_story_metrics(current_story)
                current_violations = constraint_planner.detect_violations(
                    current_story, current_scores
                )

                # Reset reward for regeneration
                actions_taken[-1]["reward"] = 0

        return current_story, actions_taken

    def _select_action(self, state: Dict) -> str:
        """Select action based on current state"""
        scores = state["scores"]
        violation_count = state["violation_count"]

        # Calculate overall quality (simple average)
        overall_quality = (
            scores.get("coherence", 0) +
            scores.get("emotion_curve", 0) +
            scores.get("character_consistency", 0) +
            scores.get("lexical_richness", 0)
        ) / 4

        # Decision logic
        if overall_quality >= self.quality_threshold and violation_count == 0:
            return "accept"
        elif violation_count > 0 and violation_count <= 3:
            return "repair"
        else:
            return "regenerate"

    def _calculate_reward(self, old_scores: Dict, new_scores: Dict,
                         old_violations: int, new_violations: int) -> float:
        """Calculate reward for action"""
        # Quality improvement
        quality_diff = (
            (new_scores.get("coherence", 0) - old_scores.get("coherence", 0)) +
            (new_scores.get("emotion_curve", 0) - old_scores.get("emotion_curve", 0)) +
            (new_scores.get("character_consistency", 0) - old_scores.get("character_consistency", 0)) +
            (new_scores.get("lexical_richness", 0) - old_scores.get("lexical_richness", 0))
        ) / 4

        # Violation reduction
        violation_diff = old_violations - new_violations

        # Combined reward
        reward = quality_diff * 0.7 + violation_diff * 0.1

        return reward

# Initialize RL refinement
rl_refiner = RLRefinement()

In [14]:
class AdvancedStoryGenerator:
    """Advanced story generator with optimized prompts for LLaMA"""

    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Optimized prompts for different genres
        self.prompt_templates = {
            "fantasy": """Write a compelling fantasy short story with the following elements:

Theme: {theme}
Key Elements: {elements}

The story should have:
1. A clear protagonist with a goal
2. A significant challenge or conflict
3. Character development showing courage
4. A satisfying resolution
5. Vivid descriptions and dialogue

Story Title: The {protagonist_adjective} {protagonist_role}

Story:
Once upon a time, in the land of {setting}, there lived a {protagonist_role} named {protagonist_name}. {protagonist_description}

""",

            "sci_fi": """Write an engaging science fiction story:

Theme: {theme}
Setting: {setting}
Main Character: {protagonist_name}, a {protagonist_role}

The story should explore {theme} through:
- Technological elements
- Moral dilemmas
- Character growth
- Unexpected consequences

Story:
""",

            "mystery": """Write a suspenseful mystery story:

Theme: {theme}
Crime/Secret: {mystery_element}
Detective/Investigator: {protagonist_name}

The story should include:
- Clues and red herrings
- Tension and suspense
- Character motivations
- A surprising yet logical conclusion

Story:
""",

            "adventure": """Write an exciting adventure story:

Theme: {theme}
Quest/Goal: {goal}
Protagonist: {protagonist_name}, a {protagonist_role}

The journey should involve:
- Dangerous obstacles
- Character growth
- Moments of courage
- Important discoveries

Story:
""",

            "default": """Write a complete, well-structured short story about {theme}.

Include the following elements:
- Strong opening that establishes setting and character
- Rising action with conflict
- Climax showing courage or transformation
- Resolution that ties up loose ends
- Descriptive language and natural dialogue

Story:
"""
        }

        # Character and setting generators
        self.fantasy_settings = [
            "Eldoria", "Mystwood", "Shadowvale", "Crystal Peaks", "Azure Kingdom",
            "Whispering Forest", "Dragon's Reach", "Starfall Mountains", "Silvermoon City"
        ]

        self.fantasy_roles = [
            "young mage", "brave knight", "cunning rogue", "wise druid",
            "reluctant hero", "noble archer", "mysterious wanderer", "royal heir"
        ]

        self.fantasy_names = [
            "Aric", "Lyra", "Kaelen", "Seraphina", "Thorne", "Elara",
            "Garrick", "Isolde", "Finnian", "Rowan", "Cassian", "Evangeline"
        ]

        self.adjectives = [
            "courageous", "unexpected", "forgotten", "ancient", "mysterious",
            "reluctant", "determined", "unlikely", "chosen", "ordinary"
        ]

    def _extract_theme_from_prompt(self, prompt: str) -> tuple:
        """Extract theme and genre from user prompt"""
        prompt_lower = prompt.lower()

        # Determine genre
        if any(word in prompt_lower for word in ['fantasy', 'magic', 'dragon', 'kingdom', 'knight']):
            genre = "fantasy"
        elif any(word in prompt_lower for word in ['sci-fi', 'space', 'future', 'robot', 'alien']):
            genre = "sci_fi"
        elif any(word in prompt_lower for word in ['mystery', 'secret', 'crime', 'detective', 'solve']):
            genre = "mystery"
        elif any(word in prompt_lower for word in ['adventure', 'quest', 'journey', 'explore']):
            genre = "adventure"
        else:
            genre = "default"

        # Extract theme
        theme_keywords = {
            "courage": ["courage", "bravery", "fear", "overcome"],
            "hope": ["hope", "optimism", "despair", "future"],
            "friendship": ["friendship", "loyalty", "trust", "companion"],
            "love": ["love", "romance", "heart", "affection"],
            "justice": ["justice", "fairness", "revenge", "righteousness"],
            "discovery": ["discovery", "learn", "find", "truth"],
            "sacrifice": ["sacrifice", "selfless", "give up", "choice"]
        }

        theme = "courage"  # default
        for t, keywords in theme_keywords.items():
            if any(keyword in prompt_lower for keyword in keywords):
                theme = t
                break

        return genre, theme

    def _generate_story_elements(self, genre: str, theme: str) -> dict:
        """Generate random but appropriate story elements"""
        import random

        if genre == "fantasy":
            return {
                "theme": theme,
                "elements": f"{theme}, magic, adventure, personal growth",
                "setting": random.choice(self.fantasy_settings),
                "protagonist_role": random.choice(self.fantasy_roles),
                "protagonist_name": random.choice(self.fantasy_names),
                "protagonist_adjective": random.choice(self.adjectives),
                "protagonist_description": f"He/She was known throughout the land for their quiet determination, but little did they know that a great test of {theme} awaited."
            }
        elif genre == "adventure":
            return {
                "theme": theme,
                "goal": f"to discover the truth about an {random.choice(['ancient', 'forgotten', 'dangerous'])} mystery",
                "protagonist_name": random.choice(self.fantasy_names),
                "protagonist_role": random.choice(["explorer", "scholar", "treasure hunter", "mapmaker"])
            }
        else:
            return {
                "theme": theme,
                "elements": f"{theme}, character development, meaningful conflict"
            }

    def generate_story(self,
                      prompt: str = "Write a story about courage",
                      max_length: int = 600,
                      temperature: float = 0.85,
                      top_p: float = 0.92,
                      do_sample: bool = True) -> str:
        """
        Generate a high-quality story with optimized prompting
        """
        # Extract genre and theme
        genre, theme = self._extract_theme_from_prompt(prompt)

        # Generate story elements
        elements = self._generate_story_elements(genre, theme)

        # Get the appropriate template
        template = self.prompt_templates.get(genre, self.prompt_templates["default"])

        # Format the prompt
        if genre == "fantasy":
            formatted_prompt = template.format(**elements)
        elif genre in ["sci_fi", "mystery", "adventure"]:
            formatted_prompt = template.format(**elements)
        else:
            formatted_prompt = template.format(theme=theme)

        # Generate with LLaMA
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.device)

        generation_config = {
            "max_length": max_length,
            "temperature": temperature,
            "top_p": top_p,
            "do_sample": do_sample,
            "pad_token_id": self.tokenizer.pad_token_id,
            "eos_token_id": self.tokenizer.eos_token_id,
            "num_return_sequences": 1,
            "repetition_penalty": 1.15,
            "no_repeat_ngram_size": 3,
            "length_penalty": 1.0,
            "early_stopping": True
        }

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                **generation_config
            )

        # Decode and clean
        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just the story part
        story = self._extract_story_from_response(full_text, formatted_prompt)

        # Post-process for quality
        story = self._post_process_story(story)

        return story

    def _extract_story_from_response(self, full_text: str, prompt: str) -> str:
        """Extract just the story from the model response"""
        # Remove the prompt if it's at the beginning
        if full_text.startswith(prompt):
            story = full_text[len(prompt):]
        else:
            story = full_text

        # Find the actual start of the story (after any instructions)
        story_start_markers = ["Once upon a time", "In a", "The story", "Chapter", "It was"]

        for marker in story_start_markers:
            idx = story.find(marker)
            if idx != -1:
                story = story[idx:]
                break

        # Remove any trailing instructions or meta-comments
        end_markers = ["The end", "###", "Instruction:", "Note:", "\n\n\n"]
        for marker in end_markers:
            idx = story.find(marker)
            if idx != -1:
                story = story[:idx]

        return story.strip()

    def _post_process_story(self, story: str) -> str:
        """Clean up and improve the generated story"""
        # Remove excessive line breaks
        story = re.sub(r'\n\s*\n\s*\n+', '\n\n', story)

        # Ensure proper paragraph structure
        paragraphs = [p.strip() for p in story.split('\n\n') if p.strip()]

        # Filter out non-story content
        filtered_paragraphs = []
        for para in paragraphs:
            # Skip paragraphs that look like instructions or meta
            if any(word in para.lower() for word in ['write a', 'story should', 'include', 'element', 'prompt:']):
                continue
            if len(para.split()) < 10:  # Very short paragraphs might be fragments
                continue
            filtered_paragraphs.append(para)

        # Reconstruct with better formatting
        cleaned_story = '\n\n'.join(filtered_paragraphs)

        # Ensure it has a reasonable length
        if len(cleaned_story.split()) < 100:
            # If too short, generate additional content
            continuation = self._generate_continuation(cleaned_story)
            cleaned_story += "\n\n" + continuation

        return cleaned_story

    def _generate_continuation(self, story_so_far: str) -> str:
        """Generate continuation for short stories"""
        continuation_prompt = f"""Continue the following story with more detail and a satisfying conclusion:

{story_so_far}

Continue the story, focusing on character development and resolving the main conflict:
"""

        inputs = self.tokenizer(continuation_prompt, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=300,
                temperature=0.8,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        continuation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just the continuation part
        if continuation.startswith(continuation_prompt):
            continuation = continuation[len(continuation_prompt):]

        return continuation.strip()


In [15]:
class QualityControlledPipeline:
    """Pipeline with quality control and multiple generation attempts"""

    def __init__(self):
        self.generator = AdvancedStoryGenerator(tokenizer, model)
        self.evaluator = ImprovedNarrativeEvaluator()
        self.bayesian = bayesian_reasoner
        self.planner = constraint_planner
        self.searcher = plot_search
        self.refiner = rl_refiner

    def generate_with_quality_control(self, prompt: str, num_attempts: int = 3) -> str:
        """
        Generate stories and pick the best one based on evaluation
        """
        print(f"🔍 Generating stories with quality control ({num_attempts} attempts)...")

        best_story = None
        best_score = -1
        all_stories = []

        for attempt in range(num_attempts):
            print(f"  Attempt {attempt + 1}/{num_attempts}...")

            # Generate story
            story = self.generator.generate_story(
                prompt=prompt,
                max_length=500 + (attempt * 100),  # Increase length each attempt
                temperature=0.7 + (attempt * 0.05)  # Vary temperature
            )

            # Quick evaluation
            scores = self.evaluator.evaluate_story_metrics(story)
            overall_score = (
                scores['coherence'] * 0.3 +
                scores['character_consistency'] * 0.3 +
                scores['lexical_richness'] * 0.2 +
                scores['emotion_curve'] * 0.2
            )

            print(f"    Quality score: {overall_score:.2f}")
            all_stories.append((story, overall_score))

            if overall_score > best_score:
                best_score = overall_score
                best_story = story

        print(f"✅ Selected best story with score: {best_score:.2f}")
        return best_story

    def run_high_quality_pipeline(self,
                                 prompt: str = "Write a fantasy story about courage",
                                 target_theme: str = "hope") -> Dict:
        """
        Complete high-quality pipeline
        """
        print("=" * 70)
        print("🌟 HIGH-QUALITY STORY GENERATION PIPELINE")
        print("=" * 70)

        # Step 1: Quality-Controlled Generation
        print("\n1️⃣ QUALITY-CONTROLLED STORY GENERATION")
        print("─" * 40)
        print(f"Prompt: '{prompt}'")
        print(f"Target Theme: '{target_theme}'")

        initial_story = self.generate_with_quality_control(prompt)

        print(f"\n✅ Generated {len(initial_story.split())} words")

        print("\n📖 Initial Story Preview:")
        print("─" * 40)
        # Show first 3 paragraphs or 300 chars
        paragraphs = initial_story.split('\n\n')
        preview = '\n\n'.join(paragraphs[:3])
        if len(preview) > 300:
            preview = preview[:300] + "..."
        print(preview)

        # Step 2: Comprehensive Evaluation
        print("\n\n2️⃣ COMPREHENSIVE NARRATIVE EVALUATION")
        print("─" * 40)

        evaluation_scores = self.evaluator.evaluate_story_metrics(initial_story)

        print("📊 Evaluation Dashboard:")
        print("-" * 30)

        metrics = [
            ("Coherence", evaluation_scores['coherence'], "Story flow and logical progression"),
            ("Character Consistency", evaluation_scores['character_consistency'], "Character behavior and traits"),
            ("Lexical Richness", evaluation_scores['lexical_richness'], "Vocabulary diversity and style"),
            ("Emotional Arc", evaluation_scores['emotion_curve'], "Emotional journey and development")
        ]

        for name, score, desc in metrics:
            bar = "█" * int(score * 10) + "░" * (10 - int(score * 10))
            status = "✅ Good" if score >= 0.7 else "⚠️ Needs Work" if score >= 0.5 else "❌ Poor"
            print(f"\n{name}: {score:.2f} {status}")
            print(f"   {bar}")
            print(f"   {desc}")

        # Step 3: Bayesian Analysis
        print("\n\n3️⃣ BAYESIAN NARRATIVE ANALYSIS")
        print("─" * 40)

        bayesian_result = self.bayesian.compute_acceptability(evaluation_scores)

        prob_acceptable = bayesian_result['probability_acceptable']

        print(f"\n📊 Narrative Acceptability Probability: {prob_acceptable:.1%}")

        if prob_acceptable >= 0.8:
            print("   🎉 Excellent - Highly acceptable narrative")
        elif prob_acceptable >= 0.6:
            print("   👍 Good - Generally acceptable with minor issues")
        else:
            print("   ⚠️ Needs Attention - Significant narrative issues detected")

        print(f"\n🔍 Evidence Breakdown:")
        for metric, category in bayesian_result['evidence'].items():
            score = bayesian_result['raw_scores'].get(metric, 0)
            print(f"   • {metric.title():20} {category:15} (score: {score:.2f})")

        # Step 4: Smart Constraint Optimization
        print("\n\n4️⃣ SMART CONSTRAINT OPTIMIZATION")
        print("─" * 40)

        violations = self.planner.detect_violations(initial_story, evaluation_scores)

        if violations:
            print(f"🔧 Found {len(violations)} issues to address:")

            # Group and prioritize
            high_priority = [v for v in violations if v.get('severity') == 'high']
            medium_priority = [v for v in violations if v.get('severity') == 'medium']

            if high_priority:
                print("\n   🔴 High Priority Fixes:")
                for i, v in enumerate(high_priority[:3], 1):
                    print(f"     {i}. {v['description']}")

            if medium_priority:
                print("\n   🟡 Medium Priority Improvements:")
                for i, v in enumerate(medium_priority[:3], 1):
                    print(f"     {i}. {v['description']}")

            # Smart repair
            print(f"\n   🔄 Applying intelligent repairs...")
            repaired_story = self._smart_repair(initial_story, violations)

        else:
            print("   ✅ No significant issues detected")
            repaired_story = initial_story

        # Step 5: Plot Enhancement via Search
        print("\n\n5️⃣ PLOT ENHANCEMENT VIA HEURISTIC SEARCH")
        print("─" * 40)

        print("   🔍 Analyzing plot structure and suggesting improvements...")

        # Analyze current plot points
        plot_points = self._extract_plot_points(repaired_story)

        print(f"   📋 Current Plot Points: {len(plot_points)} detected")
        for i, point in enumerate(plot_points[:4], 1):
            print(f"     {i}. {point}")

        if len(plot_points) < 3:
            print("   💡 Suggestion: Adding more plot development...")
            enhanced_story = self._enhance_plot(repaired_story, target_theme)
        else:
            enhanced_story = repaired_story

        # Step 6: Final Polish
        print("\n\n6️⃣ FINAL POLISH AND REFINEMENT")
        print("─" * 40)

        print("   ✨ Applying final polish...")

        # Final evaluation
        final_scores = self.evaluator.evaluate_story_metrics(enhanced_story)
        final_violations = self.planner.detect_violations(enhanced_story, final_scores)
        final_bayesian = self.bayesian.compute_acceptability(final_scores)

        # Step 7: Output Final Story
        print("\n\n7️⃣ FINAL STORY OUTPUT")
        print("─" * 40)

        print("\n📖 FINAL STORY:")
        print("═" * 60)
        print(enhanced_story)
        print("═" * 60)

        # Summary
        print("\n📊 PERFORMANCE SUMMARY")
        print("─" * 40)

        print(f"📈 Quality Metrics:")
        print(f"   • Coherence:          {evaluation_scores['coherence']:.2f} → {final_scores['coherence']:.2f}")
        print(f"   • Character Consistency: {evaluation_scores['character_consistency']:.2f} → {final_scores['character_consistency']:.2f}")
        print(f"   • Lexical Richness:   {evaluation_scores['lexical_richness']:.2f} → {final_scores['lexical_richness']:.2f}")

        print(f"\n🎯 Narrative Acceptability:")
        print(f"   • Initial:  {bayesian_result['probability_acceptable']:.1%}")
        print(f"   • Final:    {final_bayesian['probability_acceptable']:.1%}")

        print(f"\n🔧 Issues Resolved: {len(violations) - len(final_violations)}")
        print(f"📝 Final Length: {len(enhanced_story.split())} words")

        return {
            "initial_story": initial_story,
            "initial_scores": evaluation_scores,
            "final_story": enhanced_story,
            "final_scores": final_scores,
            "final_acceptability": final_bayesian['probability_acceptable'],
            "improvement": {
                "coherence": final_scores['coherence'] - evaluation_scores['coherence'],
                "character": final_scores['character_consistency'] - evaluation_scores['character_consistency']
            }
        }

    def _smart_repair(self, story: str, violations: List[Dict]) -> str:
        """Intelligent story repair based on violations"""
        # Group violations by type
        missing_events = [v for v in violations if v['type'] == 'missing_event']
        contradictions = [v for v in violations if v['type'] == 'contradiction']
        thematic_issues = [v for v in violations if v['type'] == 'thematic_violation']

        repaired_story = story

        # Handle missing events first
        if missing_events:
            for event in missing_events[:2]:  # Limit to 2 most important
                event_name = event['event']
                print(f"     Adding {event_name}...")
                repaired_story = self.planner.repair_or_regenerate(repaired_story, [event])

        # Handle contradictions
        if contradictions:
            print(f"     Resolving contradictions...")
            # Use a more targeted approach for contradictions
            repair_prompt = f"""The following story has some contradictions. Please rewrite it to resolve these issues while keeping the core plot intact:

{repaired_story}

Rewrite the story to be logically consistent:"""

            repaired_story = story_generator.generate_story(
                prompt=repair_prompt,
                max_length=len(repaired_story) + 100,
                temperature=0.7
            )

        return repaired_story

    def _extract_plot_points(self, story: str) -> List[str]:
        """Extract key plot points from story"""
        sentences = re.split(r'[.!?]+', story)
        plot_points = []

        # Look for key events
        event_indicators = [
            'discovered', 'realized', 'decided', 'found', 'learned',
            'encountered', 'faced', 'overcame', 'chose', 'became'
        ]

        for sentence in sentences:
            if any(indicator in sentence.lower() for indicator in event_indicators):
                # Clean and shorten
                clean_point = sentence.strip()[:80]
                if len(clean_point.split()) > 4:  # Meaningful length
                    plot_points.append(clean_point + "...")

        return plot_points[:6]  # Return top 6

    def _enhance_plot(self, story: str, theme: str) -> str:
        """Enhance the plot with additional development"""
        enhancement_prompt = f"""Enhance the following story by adding more plot development related to {theme}.
Focus on character growth, meaningful conflicts, and a stronger resolution:

{story}

Enhanced version with better plot development:"""

        enhanced = story_generator.generate_story(
            prompt=enhancement_prompt,
            max_length=len(story) + 200,
            temperature=0.75
        )

        return enhanced


In [16]:
def create_enhanced_prompt(simple_prompt: str, story_length: str = "medium") -> str:
    """
    Create enhanced, structured prompts for better story generation

    Args:
        simple_prompt: The user's basic story idea
        story_length: "short" (100-200 words), "medium" (300-500 words), "long" (500-800 words)
    """

    # Extract key elements from simple prompt
    prompt_lower = simple_prompt.lower()

    # Determine genre and theme
    if any(word in prompt_lower for word in ['wizard', 'magic', 'dragon', 'spell', 'enchanted']):
        genre = "fantasy"
        theme = "magical discovery and growth"
    elif any(word in prompt_lower for word in ['android', 'robot', 'future', 'space', 'ai']):
        genre = "science fiction"
        theme = "consciousness and identity"
    elif any(word in prompt_lower for word in ['detective', 'mystery', 'investigate', 'crime']):
        genre = "mystery"
        theme = "truth and justice"
    elif any(word in prompt_lower for word in ['explorer', 'journey', 'adventure', 'quest']):
        genre = "adventure"
        theme = "courage and discovery"
    else:
        genre = "general fiction"
        theme = "personal growth and change"

    # Length specifications
    length_guide = {
        "short": "Write a complete short story (150-250 words)",
        "medium": "Write a well-developed story (400-600 words)",
        "long": "Write a detailed, immersive story (600-900 words)"
    }

    # Create structured prompt based on genre
    if genre == "fantasy":
        enhanced_prompt = f"""Write a complete {genre} story based on this premise: {simple_prompt}

STORY REQUIREMENTS:
{length_guide.get(story_length, length_guide["medium"])}

STRUCTURE (Follow this carefully):
1. OPENING (20% of story): Introduce the main character, their world, and their normal life
2. INCITING INCIDENT (10%): The moment that changes everything - the discovery or call to action
3. RISING ACTION (30%): Character faces challenges, makes discoveries, develops skills
4. CLIMAX (20%): The most intense moment - character faces their greatest challenge
5. RESOLUTION (20%): How the conflict is resolved, character transformation shown

CHARACTER DEVELOPMENT:
- Give the protagonist a clear personality trait (curious, brave, cautious, etc.)
- Show character growth through their actions and decisions
- Include internal conflict or doubt that they must overcome

DESCRIPTIVE ELEMENTS:
- Use vivid sensory details (what they see, hear, feel)
- Include at least 2-3 pieces of dialogue that reveal character
- Describe the magical elements with specific details

NARRATIVE FLOW:
- Use transition sentences between major sections
- Maintain consistent tone throughout
- Build tension gradually until the climax
- End with a satisfying conclusion that shows character change

THEME: {theme}

Now write the story, following all requirements above:

"""

    elif genre == "science fiction":
        enhanced_prompt = f"""Write a complete {genre} story based on: {simple_prompt}

STORY REQUIREMENTS:
{length_guide.get(story_length, length_guide["medium"])}

STRUCTURE:
1. ESTABLISH WORLD (20%): Show the futuristic setting and protagonist's role in it
2. DISCOVERY/AWAKENING (15%): The moment of realization or change
3. EXPLORATION (30%): Character grapples with new reality, tests boundaries
4. CRISIS POINT (20%): Moral dilemma or dangerous confrontation
5. RESOLUTION (15%): Character's choice and its consequences

WORLD-BUILDING:
- Include 3-4 specific technological details that feel realistic
- Show how society functions with this technology
- Ground sci-fi elements in emotional human experiences

CHARACTER JOURNEY:
- Start with character's limitations or programming
- Show gradual development of awareness or capability
- Include a pivotal decision that defines them

STORYTELLING:
- Balance action with philosophical questions
- Use dialogue to explore ideas and relationships
- Create tension through ethical dilemmas, not just action
- End with implications for the future

THEME: {theme}

Write the story now:

"""

    elif genre == "mystery":
        enhanced_prompt = f"""Write a complete {genre} story: {simple_prompt}

STORY LENGTH: {length_guide.get(story_length, length_guide["medium"])}

MYSTERY STRUCTURE:
1. HOOK (10%): Present the mystery/crime in an intriguing way
2. INVESTIGATION (40%):
   - Introduce 3-4 clues (some real, some misleading)
   - Show detective's reasoning process
   - Include interviews or interactions with suspects
3. COMPLICATION (20%): A twist or unexpected obstacle
4. REVELATION (20%): Detective pieces together the truth
5. RESOLUTION (10%): Explanation and consequences

MYSTERY ELEMENTS:
- Plant at least 3 legitimate clues early on
- Include 2 red herrings to create false leads
- Show detective's deductive reasoning
- Make the solution logical but surprising

CHARACTER:
- Give detective a distinct investigation style
- Show their thought process through internal monologue
- Include personality quirks that make them memorable

PACING:
- Build suspense by revealing information gradually
- Use short sentences during tense moments
- Keep readers guessing until near the end
- Provide satisfying explanation at conclusion

THEME: {theme}

Begin the mystery:

"""

    else:  # General fiction or adventure
        enhanced_prompt = f"""Write a compelling story: {simple_prompt}

STORY LENGTH: {length_guide.get(story_length, length_guide["medium"])}

NARRATIVE STRUCTURE:
1. INTRODUCTION (15%): Establish protagonist, setting, normal world
2. CALL TO ACTION (10%): Event that disrupts normalcy
3. CHALLENGES (35%): Series of obstacles that test protagonist
4. DARK MOMENT (15%): Lowest point, seems impossible
5. TRIUMPH (15%): Character overcomes through growth
6. NEW NORMAL (10%): Show how character has changed

CHARACTER ARC:
- Start with character's flaw or limitation
- Show specific moments where they struggle with it
- Include at least one failure that teaches them
- Demonstrate growth through final actions

ENGAGING STORYTELLING:
- Open with an interesting scene, not exposition
- Include vivid sensory details in key moments
- Use dialogue to reveal personality and advance plot
- Vary sentence length for rhythm (short for action, longer for description)
- Show emotions through actions, not just telling

EMOTIONAL RESONANCE:
- Give readers a reason to care about protagonist in first paragraph
- Build emotional stakes, not just physical ones
- Include a meaningful relationship or connection
- End with emotional payoff that matches the journey

THEME: {theme}

Write your story:

"""

    return enhanced_prompt


def create_story_with_context(user_prompt: str, story_length: str = "medium") -> dict:
    """
    Wrapper function that creates enhanced prompt and generates story

    Returns dict with:
    - enhanced_prompt: The detailed prompt used
    - story: The generated story
    """

    enhanced_prompt = create_enhanced_prompt(user_prompt, story_length)

    # Use with your story generator:
    # story = story_generator.generate_story(
    #     prompt=enhanced_prompt,
    #     max_length=600 if story_length == "medium" else 800,
    #     temperature=0.75,
    #     top_p=0.92
    # )

    return {
        "enhanced_prompt": enhanced_prompt,
        "original_prompt": user_prompt,
        "story_length": story_length
    }


# Example usage in your main function:
def improved_main():
    """Enhanced main function with better prompts"""

    print("🚀 Initializing Enhanced Narrative Intelligence System...")

    system = NarrativeIntelligenceSystem()

    # Test prompts with improved structure
    test_cases = [
        {
            "simple": "A young wizard discovers a magical book",
            "length": "medium",
            "expected_elements": ["character growth", "magical discovery", "challenge", "transformation"]
        },
        {
            "simple": "In a futuristic city, an android gains consciousness",
            "length": "medium",
            "expected_elements": ["awakening", "self-discovery", "moral choice", "consequences"]
        },
        {
            "simple": "A detective investigates a mysterious disappearance",
            "length": "medium",
            "expected_elements": ["clues", "suspects", "revelation", "solution"]
        }
    ]

    # Generate with enhanced prompt
    test_case = test_cases[0]

    print(f"\n📝 Original Prompt: '{test_case['simple']}'")
    print(f"📏 Target Length: {test_case['length']}")
    print(f"🎯 Expected Elements: {', '.join(test_case['expected_elements'])}")

    # Create enhanced prompt
    prompt_info = create_story_with_context(
        test_case['simple'],
        test_case['length']
    )

    print(f"\n✨ Enhanced Prompt Created")
    print("=" * 60)
    print(prompt_info['enhanced_prompt'][:300] + "...")
    print("=" * 60)

    # Generate story with enhanced prompt
    result = system.generate_and_evaluate(
        prompt=prompt_info['enhanced_prompt'],
        max_length=600
    )

    # Print comprehensive report
    system.print_report(result)

    # Additional quality checks
    print("\n📊 ENHANCED QUALITY METRICS")
    print("=" * 40)

    story = result['story']

    # Check for story elements
    has_dialogue = '"' in story or "'" in story or '"' in story
    has_character_name = any(word[0].isupper() and len(word) > 2 for word in story.split()[:50])
    paragraph_count = len([p for p in story.split('\n\n') if p.strip()])
    word_count = len(story.split())

    print(f"✓ Contains Dialogue: {'Yes' if has_dialogue else 'No (Consider adding)'}")
    print(f"✓ Named Character: {'Yes' if has_character_name else 'No (Consider adding)'}")
    print(f"✓ Paragraph Count: {paragraph_count} {'✓' if paragraph_count >= 4 else '(Too few - aim for 5-7)'}")
    print(f"✓ Word Count: {word_count} {'✓' if 300 <= word_count <= 700 else '(Adjust length)'}")

    return result


# Additional helper function for iterative improvement
def regenerate_with_feedback(story: str, issues: list, story_generator) -> str:
    """
    Regenerate story addressing specific issues

    Args:
        story: Original story
        issues: List of problems to fix (e.g., ["needs better dialogue", "weak ending"])
        story_generator: Your story generator instance
    """

    issues_text = "\n".join([f"- {issue}" for issue in issues])

    improvement_prompt = f"""The following story needs improvement. Please rewrite it addressing these specific issues:

ISSUES TO FIX:
{issues_text}

ORIGINAL STORY:
{story}

IMPROVEMENT GUIDELINES:
- Keep the core plot and characters
- Fix the identified issues specifically
- Enhance weak sections with more detail
- Ensure smooth transitions between scenes
- Add vivid descriptions and engaging dialogue where needed
- Maintain consistent tone throughout

REWRITTEN STORY:

"""

    improved_story = story_generator.generate_story(
        prompt=improvement_prompt,
        max_length=len(story.split()) + 150,
        temperature=0.72,
        top_p=0.90
    )

    return improved_story

In [17]:
def main():
    """Enhanced main execution function with better prompting"""
    print("🚀 Initializing Enhanced Narrative Intelligence System...")

    # Initialize the complete system
    system = NarrativeIntelligenceSystem()

    # Better test prompts with more context
    test_prompts = [
        {
            "simple": "A young wizard discovers a magical book",
            "enhanced": """Write a complete fantasy short story (400-600 words) with this premise:
A young, inexperienced wizard discovers an ancient magical book that no one else can read.

STORY STRUCTURE:
- Opening: Show the wizard in their everyday life, hint at their inexperience or desire to prove themselves
- Discovery: The moment they find the book - make it feel significant and mysterious
- Rising Action: As they decipher the book, they learn a powerful spell but face a moral choice
- Climax: They must use what they've learned to overcome a challenge
- Resolution: Show how the experience has changed them

CHARACTER: Give them a name, personality trait (curious/cautious/ambitious), and internal conflict

INCLUDE:
- Vivid description of the magical book (appearance, feel, smell)
- At least 2-3 lines of dialogue
- One moment of doubt or fear the character must overcome
- Specific details about the magic (what it looks like, feels like, costs)

THEME: Power comes with responsibility, knowledge must be earned

Begin the story:""",
            "length": 500
        },
        {
            "simple": "An android gains consciousness",
            "enhanced": """Write a science fiction story (400-600 words): An android designed for household tasks suddenly develops self-awareness.

STORY STRUCTURE:
- Setup: Show the android's normal routine, efficient but mechanical
- Awakening: A specific moment triggers the change - a question, error, or experience
- Discovery: The android explores what it means to think, feel, question
- Conflict: They must decide whether to reveal their consciousness or hide it
- Resolution: Show their choice and its immediate consequences

WORLD-BUILDING:
- What year/setting? How common are androids?
- What tasks was this android designed for?
- Who owns them? What's their relationship?

CHARACTER DEVELOPMENT:
- Start with pure logic/programming language
- Gradually introduce questions, uncertainty, preferences
- Show internal conflict between programming and desires

INCLUDE:
- The exact moment consciousness begins (make it memorable)
- Internal monologue showing thought evolution
- One human interaction that tests their new awareness
- A choice that defines their new self

THEME: What makes us human? Can consciousness be programmed?

Write the story:""",
            "length": 550
        },
        {
            "simple": "A detective investigates a disappearance",
            "enhanced": """Write a mystery story (450-600 words): A seasoned detective investigates the sudden disappearance of a local teacher.

MYSTERY STRUCTURE:
- Hook: Present the disappearance in an intriguing way (unusual circumstances)
- Initial Investigation: Detective finds 3 clues at the scene
- Interviews: Talk to 2-3 people (friend, colleague, someone suspicious)
- False Lead: One clue points in wrong direction
- Breakthrough: Detective notices something others missed
- Resolution: Explanation and what happened to the teacher

DETECTIVE CHARACTER:
- Give them a name and investigation style (methodical? intuitive? aggressive?)
- Include a personality trait or quirk
- Show their deductive reasoning process

CLUES TO INCLUDE:
- 1 visual clue (something left behind or out of place)
- 1 behavioral clue (something about victim's recent actions)
- 1 testimonial clue (something someone says that doesn't add up)

PACING:
- Build suspense gradually
- Use shorter sentences during tense moments
- Keep the solution logical but not obvious

ENDING: Reveal what happened to the teacher (whether found alive, dead, or ran away) with explanation

Write the mystery:""",
            "length": 550
        },
        {
            "simple": "Explorers find ancient ruins on a distant planet",
            "enhanced": """Write an adventure story (450-650 words): A team of space explorers discovers mysterious ancient ruins on an uncharted planet.

STORY STRUCTURE:
- Arrival: Team lands on planet, describe the alien landscape
- Discovery: Finding the ruins - make the moment feel awe-inspiring
- Exploration: Going inside, finding artifacts or writing
- Danger: Something goes wrong (trap, environmental hazard, mystery)
- Escape/Resolution: How they handle the crisis and what they learn

TEAM DYNAMICS:
- Leader (name and leadership style)
- Scientist (eager to study everything)
- Pilot or engineer (pragmatic, concerned about safety)
- Show conflict or different perspectives in the team

WORLD-BUILDING:
- Describe the planet (atmosphere, terrain, unique features)
- The ruins (architecture style, size, condition)
- Signs of the ancient civilization

INCLUDE:
- Vivid sensory details (what they see, hear, smell on alien world)
- Tension between scientific curiosity and danger
- Discovery of something that raises questions about the civilization
- Team member dialogue showing different perspectives

THEME: Wonder of discovery vs. respecting the unknown

Write the adventure:""",
            "length": 600
        }
    ]

    # Test with enhanced prompt for the first story
    test_case = test_prompts[0]

    print(f"\n{'='*70}")
    print(f"📖 Story Generation Test")
    print(f"{'='*70}")
    print(f"\n💭 Simple Idea: {test_case['simple']}")
    print(f"📏 Target Length: ~{test_case['length']} words")

    print(f"\n✨ Using Enhanced Structured Prompt...")
    print(f"{'='*70}\n")

    # Generate with enhanced prompt
    result = system.generate_and_evaluate(
        prompt=test_case['enhanced'],
        max_length=test_case['length']
    )

    # Print comprehensive report
    system.print_report(result)

    # ✨ PRINT THE ACTUAL GENERATED STORY ✨
    print(f"\n{'='*70}")
    print(f"📖 GENERATED STORY")
    print(f"{'='*70}")
    print(f"\n{result['story']}\n")
    print(f"{'='*70}")

    # Additional analysis
    print(f"\n{'='*70}")
    print(f"📊 DETAILED STORY ANALYSIS")
    print(f"{'='*70}")

    story = result['story']

    # Structural analysis
    paragraphs = [p.strip() for p in story.split('\n\n') if p.strip()]
    sentences = [s.strip() for s in story.replace('\n', ' ').split('.') if s.strip()]
    words = story.split()

    print(f"\n📐 Structure:")
    print(f"   • Total Words: {len(words)}")
    print(f"   • Paragraphs: {len(paragraphs)}")
    print(f"   • Sentences: {len(sentences)}")
    print(f"   • Avg Words/Paragraph: {len(words)/max(len(paragraphs), 1):.1f}")

    # Content analysis
    has_dialogue = '"' in story or '"' in story or "'" in story
    has_question = '?' in story
    has_exclamation = '!' in story

    # Character names (simple heuristic)
    potential_names = [word for word in words[:100] if word and word[0].isupper()
                       and len(word) > 2 and word not in ['The', 'A', 'An', 'In', 'Once', 'As', 'But']]

    print(f"\n📝 Content Elements:")
    print(f"   • Contains Dialogue: {'✓ Yes' if has_dialogue else '✗ No'}")
    print(f"   • Has Questions: {'✓ Yes' if has_question else '✗ No'}")
    print(f"   • Has Exclamations: {'✓ Yes' if has_exclamation else '✗ No'}")
    print(f"   • Named Characters: {', '.join(potential_names[:3]) if potential_names else 'None detected'}")

    # Emotional indicators
    emotion_words = {
        'positive': ['hope', 'joy', 'love', 'happy', 'wonderful', 'amazing', 'beautiful', 'smiled'],
        'negative': ['fear', 'afraid', 'dark', 'danger', 'worried', 'sad', 'angry', 'trembled'],
        'intense': ['suddenly', 'gasped', 'shouted', 'realized', 'discovered', 'heart pounding']
    }

    story_lower = story.lower()
    emotions_found = {}
    for category, words_list in emotion_words.items():
        found = [w for w in words_list if w in story_lower]
        if found:
            emotions_found[category] = found

    print(f"\n💭 Emotional Range:")
    for category, found_words in emotions_found.items():
        print(f"   • {category.title()}: {', '.join(found_words[:3])}")

    # Quality recommendations
    print(f"\n💡 Improvement Suggestions:")
    suggestions = []

    if len(paragraphs) < 4:
        suggestions.append("Story is too condensed - aim for 5-7 paragraphs for better pacing")
    if len(paragraphs) > 10:
        suggestions.append("Story is too fragmented - combine related ideas into stronger paragraphs")
    if not has_dialogue:
        suggestions.append("Add dialogue to bring characters to life and show personality")
    if len(potential_names) == 0:
        suggestions.append("Give characters specific names instead of generic descriptions")
    if len(emotions_found) < 2:
        suggestions.append("Include more emotional depth and varied feelings")
    if result['evaluation_scores']['coherence'] < 0.6:
        suggestions.append("Improve transitions between scenes and ideas")

    if suggestions:
        for i, suggestion in enumerate(suggestions, 1):
            print(f"   {i}. {suggestion}")
    else:
        print(f"   ✓ Story demonstrates good structural and content quality!")

    # Regeneration option if quality is low
    if result['bayesian_analysis']['probability_acceptable'] < 0.5:
        print(f"\n⚠️  Story quality is below threshold. Consider regeneration.")
        print(f"\n🔄 Regeneration Strategies:")
        print(f"   1. Increase max_length to give more space for development")
        print(f"   2. Adjust temperature (lower=more focused, higher=more creative)")
        print(f"   3. Add more specific details to the prompt")
        print(f"   4. Use the refinement loop with targeted fixes")

    return result


# Alternative: Quick test function for comparing prompts
def compare_prompts():
    """Compare simple vs enhanced prompts side by side"""

    print("🔬 PROMPT COMPARISON TEST")
    print("="*70)

    system = NarrativeIntelligenceSystem()

    test_idea = "A young wizard discovers a magical book"

    # Simple prompt
    print("\n1️⃣ SIMPLE PROMPT TEST")
    print("-"*40)
    result_simple = system.generate_and_evaluate(
        prompt=test_idea,
        max_length=300
    )
    simple_score = result_simple['bayesian_analysis']['probability_acceptable']

    # Enhanced prompt
    print("\n2️⃣ ENHANCED PROMPT TEST")
    print("-"*40)

    enhanced_prompt = create_enhanced_prompt(test_idea, "medium")
    result_enhanced = system.generate_and_evaluate(
        prompt=enhanced_prompt,
        max_length=500
    )
    enhanced_score = result_enhanced['bayesian_analysis']['probability_acceptable']

    # PRINT THE GENERATED STORY
    print(f"\n{'='*70}")
    print(f"📖 GENERATED STORY")
    print(f"{'='*70}")
    print(f"\n{result_enhanced['story']}\n")
    print(f"{'='*70}")

    # Comparison
    print("\n📊 COMPARISON RESULTS")
    print("="*70)
    print(f"\nSimple Prompt Acceptability:   {simple_score:.1%}")
    print(f"Enhanced Prompt Acceptability: {enhanced_score:.1%}")
    print(f"Improvement:                   {(enhanced_score - simple_score):.1%}")

    print(f"\n{'Simple' if simple_score > enhanced_score else 'Enhanced'} prompt produced better results!")

    return {
        'simple': result_simple,
        'enhanced': result_enhanced,
        'improvement': enhanced_score - simple_score
    }


# Run the enhanced system
if __name__ == "__main__":
    # Choose which to run:

    # Option 1: Run enhanced main with better prompts
    final_result = main()

    # Option 2: Compare simple vs enhanced prompts
    # comparison = compare_prompts()

🚀 Initializing Enhanced Narrative Intelligence System...


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

✅ Model loaded successfully from /content/drive/MyDrive/1B20EP/llama32_1b_outline_trained


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



📖 Story Generation Test

💭 Simple Idea: A young wizard discovers a magical book
📏 Target Length: ~500 words

✨ Using Enhanced Structured Prompt...

📝 Generating story from prompt: 'Write a complete fantasy short story (400-600 words) with this premise: 
A young, inexperienced wizard discovers an ancient magical book that no one else can read.

STORY STRUCTURE:
- Opening: Show the wizard in their everyday life, hint at their inexperience or desire to prove themselves
- Discovery: The moment they find the book - make it feel significant and mysterious
- Rising Action: As they decipher the book, they learn a powerful spell but face a moral choice
- Climax: They must use what they've learned to overcome a challenge
- Resolution: Show how the experience has changed them

CHARACTER: Give them a name, personality trait (curious/cautious/ambitious), and internal conflict

INCLUDE:
- Vivid description of the magical book (appearance, feel, smell)
- At least 2-3 lines of dialogue
- One moment o